Instalando Pacote

Puxando Dataset de Discurso de Ódio

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import pandas as pd
# import transformers
# from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from transformers import AdamW
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer



# link: https://huggingface.co/datasets/ag_news

In [3]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/t5_key_text/original/base_offensive_test.csv', sep = ';', decimal = ',')
test.head()

,text,label
0,#ibelieveblaseyford is liar she is fat ugly li...,1
1,@user @user @user I got in a pretty deep debat...,0
2,"...if you want more shootings and more death, ...",0
3,Angels now have 6 runs. Five of them have come...,0
4,#Travel #Movies and Unix #Fortune combined Vi...,0


In [4]:
# X_train = dataset['train']['text']
X_test = test['text']
# y_train = dataset['train']['label']
y_test = test['label']

In [5]:
nclasses = len(np.unique(y_test))

In [6]:
nclasses

2

In [7]:
def puxa_dados_normal(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [8]:
def puxa_dados_normal_pkl(caminho):
  csv = pd.read_pickle(caminho)
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [9]:
def puxa_original(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')

  X = csv['texto'].values
  y = csv['label'].values


  return X, y

In [10]:
def puxa_dados_lambada(caminho, original):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = pd.read_csv(original, sep = ';', decimal = ',')

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [11]:
def desempenho_LSTM(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [12]:
def desempenho_CNN(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Conv1D(filters=100, kernel_size = 3, activation = 'relu',
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),

        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu',
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [13]:

y_test = np.array(y_test)


# Original

In [14]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_original(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30
6/6 - 14s - loss: 0.6757 - accuracy: 0.6588 - 14s/epoch - 2s/step
Epoch 2/30
6/6 - 1s - loss: 0.6504 - accuracy: 0.6882 - 1s/epoch - 226ms/step
Epoch 3/30
6/6 - 1s - loss: 0.5979 - accuracy: 0.6882 - 1s/epoch - 191ms/step
Epoch 4/30
6/6 - 1s - loss: 0.6091 - accuracy: 0.6706 - 1s/epoch - 230ms/step
Epoch 5/30
6/6 - 1s - loss: 0.5504 - accuracy: 0.7176 - 1s/epoch - 191ms/step
Epoch 6/30
6/6 - 1s - loss: 0.5305 - accuracy: 0.7706 - 1s/epoch - 192ms/step
Epoch 7/30
6/6 - 1s - loss: 0.4851 - accuracy: 0.7882 - 1s/epoch - 193ms/step
Epoch 8/30
6/6 - 1s - loss: 0.4513 - accuracy: 0.7471 - 1s/epoch - 194ms/step
Epoch 9/30
6/6 - 1s - loss: 0.3907 - accuracy: 0.7941 - 1s/epoch - 231ms/step
Epoch 10/30
6/6 - 2s - loss: 0.3714 - accuracy: 0.7882 - 2s/epoch - 321ms/step
Epoch 11/30
6/6 - 2s - loss: 0.3590 - accuracy: 0.7824 - 2s/epoch - 261ms/step
Epoch 12/30
6/6 - 1s - loss: 0.3307 - accuracy: 0.7706 - 931ms/epoch - 155ms/step
Epoch 13/30
6/6 - 1s - loss: 0.2785 - accuracy: 0.9235 - 

In [15]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [16]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.711628,0.423468,0.609302,0.545600
1,original2,0.577907,0.499165,0.493023,0.475313
2,original3,0.708140,0.443590,0.696512,0.551510
3,original4,0.712791,0.445655,0.593023,0.546293
4,original5,0.716279,0.457447,0.634884,0.557311


In [17]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.685349,0.453865,0.605349,0.535206
std,0.060132,0.028115,0.074107,0.033810
min,0.577907,0.423468,0.493023,0.475313
25%,0.708140,0.443590,0.593023,0.545600
50%,0.711628,0.445655,0.609302,0.546293
75%,0.712791,0.457447,0.634884,0.551510
max,0.716279,0.499165,0.696512,0.557311


# ProtAugment

In [18]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/protaugment/base_offensive_'+str(perc)+'_protaugment'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/protaugment/base_offensive_'+str(perc)+'_protaugment'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/protaugment/base_offensive_'+str(perc)+'_protaugment'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/protaugment/base_offensive_'+str(perc)+'_protaugment'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/protaugment/base_offensive_'+str(perc)+'_protaugment'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet', 4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
30/30 - 8s - loss: 0.6438 - accuracy: 0.6719 - 8s/epoch - 265ms/step
Epoch 2/30
30/30 - 5s - loss: 0.5776 - accuracy: 0.6771 - 5s/epoch - 150ms/step
Epoch 3/30
30/30 - 7s - loss: 0.4604 - accuracy: 0.6771 - 7s/epoch - 218ms/step
Epoch 4/30
30/30 - 6s - loss: 0.3272 - accuracy: 0.7158 - 6s/epoch - 199ms/step
Epoch 5/30
30/30 - 3s - loss: 0.2135 - accuracy: 0.9028 - 3s/epoch - 86ms/step
Epoch 6/30
30/30 - 2s - loss: 0.1141 - accuracy: 0.9175 - 2s/epoch - 83ms/step
Epoch 7/30
30/30 - 2s - loss: 0.1362 - accuracy: 0.9080 - 2s/epoch - 60ms/step
Epoch 8/30
30/30 - 2s - loss: 0.1082 - accuracy: 0.9248 - 2s/epoch - 80ms/step
Epoch 9/30
30/30 - 1s - loss: 0.1193 - accuracy: 0.9091 - 819ms/epoch - 27ms/step
Epoch 10/30
30/30 - 1s - loss: 0.1139 - accuracy: 0.9101 - 1s/epoch - 35ms/step
Epoch 11/30
30/30 - 1s - loss: 0.1159 - accuracy: 0.9206 - 894ms/epoch - 30ms/step
Epoch 12/30
30/30 - 1s - loss: 0.1029 - accuracy: 0.9227 - 820ms/epoch - 27ms/step
Epoch 13/30
30/30 - 0s - loss: 0.097

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


30/30 - 9s - loss: 0.6462 - accuracy: 0.6681 - 9s/epoch - 311ms/step
Epoch 2/30
30/30 - 5s - loss: 0.5424 - accuracy: 0.6702 - 5s/epoch - 182ms/step
Epoch 3/30
30/30 - 3s - loss: 0.3793 - accuracy: 0.6702 - 3s/epoch - 95ms/step
Epoch 4/30
30/30 - 4s - loss: 0.2793 - accuracy: 0.9204 - 4s/epoch - 134ms/step
Epoch 5/30
30/30 - 4s - loss: 0.2033 - accuracy: 0.9770 - 4s/epoch - 149ms/step
Epoch 6/30
30/30 - 2s - loss: 0.1082 - accuracy: 0.9812 - 2s/epoch - 65ms/step
Epoch 7/30
30/30 - 2s - loss: 0.0907 - accuracy: 0.9791 - 2s/epoch - 63ms/step
Epoch 8/30
30/30 - 2s - loss: 0.0778 - accuracy: 0.9885 - 2s/epoch - 57ms/step
Epoch 9/30
30/30 - 1s - loss: 0.0610 - accuracy: 0.9895 - 1s/epoch - 42ms/step
Epoch 10/30
30/30 - 1s - loss: 0.0616 - accuracy: 0.9822 - 1s/epoch - 35ms/step
Epoch 11/30
30/30 - 2s - loss: 0.0672 - accuracy: 0.9812 - 2s/epoch - 50ms/step
Epoch 12/30
30/30 - 1s - loss: 0.0656 - accuracy: 0.9906 - 942ms/epoch - 31ms/step
Epoch 13/30
30/30 - 1s - loss: 0.0590 - accuracy: 0.9

...

30/30 - 10s - loss: 0.6416 - accuracy: 0.6757 - 10s/epoch - 339ms/step
Epoch 2/30
30/30 - 4s - loss: 0.5421 - accuracy: 0.6966 - 4s/epoch - 131ms/step
Epoch 3/30
30/30 - 6s - loss: 0.2719 - accuracy: 0.8989 - 6s/epoch - 204ms/step
Epoch 4/30
30/30 - 4s - loss: 0.1079 - accuracy: 0.9187 - 4s/epoch - 124ms/step
Epoch 5/30
30/30 - 3s - loss: 0.0767 - accuracy: 0.9562 - 3s/epoch - 109ms/step
Epoch 6/30
30/30 - 2s - loss: 0.0772 - accuracy: 0.9875 - 2s/epoch - 58ms/step
Epoch 7/30
30/30 - 1s - loss: 0.0838 - accuracy: 0.9917 - 849ms/epoch - 28ms/step
Epoch 8/30
30/30 - 1s - loss: 0.0497 - accuracy: 0.9948 - 1s/epoch - 50ms/step
Epoch 9/30
30/30 - 2s - loss: 0.0562 - accuracy: 0.9927 - 2s/epoch - 73ms/step
Epoch 10/30
30/30 - 2s - loss: 0.0575 - accuracy: 0.9937 - 2s/epoch - 57ms/step
Epoch 11/30
30/30 - 1s - loss: 0.0593 - accuracy: 0.9864 - 1s/epoch - 48ms/step
Epoch 12/30
30/30 - 1s - loss: 0.0472 - accuracy: 0.9969 - 1s/epoch - 43ms/step
Epoch 13/30
30/30 - 1s - loss: 0.0484 - accuracy: 

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
31/31 - 9s - loss: 0.6465 - accuracy: 0.6677 - 9s/epoch - 282ms/step
Epoch 2/30
31/31 - 6s - loss: 0.5251 - accuracy: 0.7109 - 6s/epoch - 205ms/step
Epoch 3/30
31/31 - 5s - loss: 0.3065 - accuracy: 0.8807 - 5s/epoch - 158ms/step
Epoch 4/30
31/31 - 4s - loss: 0.1343 - accuracy: 0.9424 - 4s/epoch - 128ms/step
Epoch 5/30
31/31 - 2s - loss: 0.1038 - accuracy: 0.9486 - 2s/epoch - 63ms/step
Epoch 6/30
31/31 - 3s - loss: 0.1019 - accuracy: 0.9403 - 3s/epoch - 107ms/step
Epoch 7/30
31/31 - 2s - loss: 0.0831 - accuracy: 0.9516 - 2s/epoch - 68ms/step
Epoch 8/30
31/31 - 1s - loss: 0.0844 - accuracy: 0.9568 - 1s/epoch - 34ms/step
Epoch 9/30
31/31 - 1s - loss: 0.0781 - accuracy: 0.9599 - 1s/epoch - 34ms/step
Epoch 10/30
31/31 - 2s - loss: 0.0858 - accuracy: 0.9578 - 2s/epoch - 56ms/step
Epoch 11/30
31/31 - 2s - loss: 0.0812 - accuracy: 0.9630 - 2s/epoch - 63ms/step
Epoch 12/30
31/31 - 1s - loss: 0.0831 - accuracy: 0.9547 - 619ms/epoch - 20ms/step
Epoch 13/30
31/31 - 1s - loss: 0.0826 - a

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


30/30 - 10s - loss: 0.6414 - accuracy: 0.6638 - 10s/epoch - 344ms/step
Epoch 2/30
30/30 - 5s - loss: 0.5249 - accuracy: 0.6702 - 5s/epoch - 173ms/step
Epoch 3/30
30/30 - 5s - loss: 0.3694 - accuracy: 0.6998 - 5s/epoch - 164ms/step
Epoch 4/30
30/30 - 4s - loss: 0.2181 - accuracy: 0.9450 - 4s/epoch - 142ms/step
Epoch 5/30
30/30 - 3s - loss: 0.1402 - accuracy: 0.9789 - 3s/epoch - 110ms/step
Epoch 6/30
30/30 - 2s - loss: 0.1274 - accuracy: 0.9841 - 2s/epoch - 66ms/step
Epoch 7/30
30/30 - 2s - loss: 0.1122 - accuracy: 0.9894 - 2s/epoch - 58ms/step
Epoch 8/30
30/30 - 3s - loss: 0.1014 - accuracy: 0.9947 - 3s/epoch - 96ms/step
Epoch 9/30
30/30 - 2s - loss: 0.1022 - accuracy: 0.9894 - 2s/epoch - 82ms/step
Epoch 10/30
30/30 - 3s - loss: 0.1041 - accuracy: 0.9926 - 3s/epoch - 92ms/step
Epoch 11/30
30/30 - 1s - loss: 0.0972 - accuracy: 0.9937 - 635ms/epoch - 21ms/step
Epoch 12/30
30/30 - 1s - loss: 0.0959 - accuracy: 0.9873 - 895ms/epoch - 30ms/step
Epoch 13/30
30/30 - 1s - loss: 0.1020 - accurac

In [19]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [20]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.642093,0.523787,0.600000,0.544524
std,0.042033,0.021071,0.062499,0.034125
min,0.570930,0.507920,0.522093,0.503366
25%,0.640698,0.508683,0.547674,0.511174
50%,0.659302,0.519647,0.617442,0.568741
75%,0.660465,0.523140,0.647674,0.569075
max,0.679070,0.559544,0.665116,0.570266


# EDA

In [21]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/eda/base_offensive_'+str(perc)+'_eda'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/eda/base_offensive_'+str(perc)+'_eda'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/eda/base_offensive_'+str(perc)+'_eda'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/eda/base_offensive_'+str(perc)+'_eda'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/eda/base_offensive_'+str(perc)+'_eda'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 10s - loss: 0.6522 - accuracy: 0.6588 - 10s/epoch - 356ms/step
Epoch 2/30
27/27 - 5s - loss: 0.5912 - accuracy: 0.6706 - 5s/epoch - 196ms/step
Epoch 3/30
27/27 - 7s - loss: 0.4793 - accuracy: 0.6706 - 7s/epoch - 249ms/step
Epoch 4/30
27/27 - 5s - loss: 0.3759 - accuracy: 0.6706 - 5s/epoch - 176ms/step
Epoch 5/30
27/27 - 3s - loss: 0.2936 - accuracy: 0.9118 - 3s/epoch - 129ms/step
Epoch 6/30
27/27 - 5s - loss: 0.2172 - accuracy: 0.9447 - 5s/epoch - 175ms/step
Epoch 7/30
27/27 - 3s - loss: 0.1674 - accuracy: 0.9576 - 3s/epoch - 97ms/step
Epoch 8/30
27/27 - 2s - loss: 0.1549 - accuracy: 0.9553 - 2s/epoch - 63ms/step
Epoch 9/30
27/27 - 2s - loss: 0.1548 - accuracy: 0.9471 - 2s/epoch - 70ms/step
Epoch 10/30
27/27 - 1s - loss: 0.1420 - accuracy: 0.9529 - 1s/epoch - 51ms/step
Epoch 11/30
27/27 - 2s - loss: 0.1284 - accuracy: 0.9588 - 2s/epoch - 60ms/step
Epoch 12/30
27/27 - 1s - loss: 0.1246 - accuracy: 0.9671 - 1s/epoch - 51ms/step
Epoch 13/30
27/27 - 1s - loss: 0.1246 - accuracy: 0.

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 10s - loss: 0.6593 - accuracy: 0.6525 - 10s/epoch - 354ms/step
Epoch 2/30
27/27 - 5s - loss: 0.6167 - accuracy: 0.6702 - 5s/epoch - 196ms/step
Epoch 3/30
27/27 - 4s - loss: 0.5524 - accuracy: 0.6855 - 4s/epoch - 131ms/step
Epoch 4/30
27/27 - 6s - loss: 0.4101 - accuracy: 0.8139 - 6s/epoch - 204ms/step
Epoch 5/30
27/27 - 4s - loss: 0.2537 - accuracy: 0.8999 - 4s/epoch - 136ms/step
Epoch 6/30
27/27 - 3s - loss: 0.2101 - accuracy: 0.9152 - 3s/epoch - 93ms/step
Epoch 7/30
27/27 - 2s - loss: 0.1780 - accuracy: 0.9211 - 2s/epoch - 92ms/step
Epoch 8/30
27/27 - 3s - loss: 0.1941 - accuracy: 0.9164 - 3s/epoch - 124ms/step
Epoch 9/30
27/27 - 1s - loss: 0.1696 - accuracy: 0.9282 - 1s/epoch - 43ms/step
Epoch 10/30
27/27 - 1s - loss: 0.1751 - accuracy: 0.9105 - 908ms/epoch - 34ms/step
Epoch 11/30
27/27 - 2s - loss: 0.1705 - accuracy: 0.9258 - 2s/epoch - 70ms/step
Epoch 12/30
27/27 - 2s - loss: 0.1560 - accuracy: 0.9282 - 2s/epoch - 83ms/step
Epoch 13/30
27/27 - 2s - loss: 0.1741 - accuracy:

...

Epoch 1/30
27/27 - 9s - loss: 0.6367 - accuracy: 0.6588 - 9s/epoch - 338ms/step
Epoch 2/30
27/27 - 6s - loss: 0.5822 - accuracy: 0.6706 - 6s/epoch - 211ms/step
Epoch 3/30
27/27 - 5s - loss: 0.4876 - accuracy: 0.6706 - 5s/epoch - 173ms/step
Epoch 4/30
27/27 - 4s - loss: 0.3750 - accuracy: 0.6706 - 4s/epoch - 142ms/step
Epoch 5/30
27/27 - 2s - loss: 0.3096 - accuracy: 0.8776 - 2s/epoch - 83ms/step
Epoch 6/30
27/27 - 3s - loss: 0.2818 - accuracy: 0.9212 - 3s/epoch - 105ms/step
Epoch 7/30
27/27 - 3s - loss: 0.2511 - accuracy: 0.9600 - 3s/epoch - 99ms/step
Epoch 8/30
27/27 - 2s - loss: 0.2395 - accuracy: 0.9541 - 2s/epoch - 83ms/step
Epoch 9/30
27/27 - 2s - loss: 0.2370 - accuracy: 0.9459 - 2s/epoch - 92ms/step
Epoch 10/30
27/27 - 2s - loss: 0.2184 - accuracy: 0.9600 - 2s/epoch - 65ms/step
Epoch 11/30
27/27 - 2s - loss: 0.2060 - accuracy: 0.9671 - 2s/epoch - 57ms/step
Epoch 12/30
27/27 - 2s - loss: 0.2200 - accuracy: 0.9459 - 2s/epoch - 78ms/step
Epoch 13/30
27/27 - 2s - loss: 0.2040 - accu

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
27/27 - 7s - loss: 0.6677 - accuracy: 0.6635 - 7s/epoch - 276ms/step
Epoch 2/30
27/27 - 7s - loss: 0.5749 - accuracy: 0.6706 - 7s/epoch - 251ms/step
Epoch 3/30
27/27 - 4s - loss: 0.4166 - accuracy: 0.7271 - 4s/epoch - 131ms/step
Epoch 4/30
27/27 - 3s - loss: 0.2038 - accuracy: 0.8400 - 3s/epoch - 122ms/step
Epoch 5/30
27/27 - 3s - loss: 0.1409 - accuracy: 0.9729 - 3s/epoch - 114ms/step
Epoch 6/30
27/27 - 4s - loss: 0.1254 - accuracy: 0.9859 - 4s/epoch - 151ms/step
Epoch 7/30
27/27 - 3s - loss: 0.1205 - accuracy: 0.9929 - 3s/epoch - 106ms/step
Epoch 8/30
27/27 - 2s - loss: 0.1037 - accuracy: 0.9894 - 2s/epoch - 65ms/step
Epoch 9/30
27/27 - 2s - loss: 0.1000 - accuracy: 1.0000 - 2s/epoch - 56ms/step
Epoch 10/30
27/27 - 2s - loss: 0.1107 - accuracy: 0.9965 - 2s/epoch - 58ms/step
Epoch 11/30
27/27 - 2s - loss: 0.1016 - accuracy: 0.9953 - 2s/epoch - 74ms/step
Epoch 12/30
27/27 - 2s - loss: 0.0967 - accuracy: 0.9988 - 2s/epoch - 63ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0848 - ac

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 7s - loss: 0.6539 - accuracy: 0.6659 - 7s/epoch - 263ms/step
Epoch 2/30
27/27 - 6s - loss: 0.5863 - accuracy: 0.6706 - 6s/epoch - 231ms/step
Epoch 3/30
27/27 - 5s - loss: 0.4388 - accuracy: 0.7341 - 5s/epoch - 170ms/step
Epoch 4/30
27/27 - 3s - loss: 0.2686 - accuracy: 0.8824 - 3s/epoch - 129ms/step
Epoch 5/30
27/27 - 4s - loss: 0.1274 - accuracy: 0.9718 - 4s/epoch - 145ms/step
Epoch 6/30
27/27 - 2s - loss: 0.0750 - accuracy: 0.9800 - 2s/epoch - 67ms/step
Epoch 7/30
27/27 - 1s - loss: 0.0642 - accuracy: 0.9847 - 1s/epoch - 54ms/step
Epoch 8/30
27/27 - 2s - loss: 0.0519 - accuracy: 0.9847 - 2s/epoch - 71ms/step
Epoch 9/30
Epoch 10/30
27/27 - 2s - loss: 0.0363 - accuracy: 0.9871 - 2s/epoch - 79ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0488 - accuracy: 0.9871 - 1s/epoch - 52ms/step
Epoch 12/30
27/27 - 3s - loss: 0.0461 - accuracy: 0.9906 - 3s/epoch - 115ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0405 - accuracy: 0.9847 - 760ms/epoch - 28ms/step
Epoch 14/30
27/27 - 1s - loss: 0.0340 -

In [22]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [23]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.675581,0.491987,0.531395,0.507082
1,eda2,0.555814,0.504811,0.547674,0.509816
2,eda3,0.666279,0.508748,0.675581,0.569437
3,eda4,0.640698,0.524594,0.533721,0.518889
4,eda5,0.593023,0.506557,0.626744,0.553156


In [24]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.626279,0.507340,0.583023,0.531676
std,0.050769,0.011645,0.064854,0.027989
min,0.555814,0.491987,0.531395,0.507082
25%,0.593023,0.504811,0.533721,0.509816
50%,0.640698,0.506557,0.547674,0.518889
75%,0.666279,0.508748,0.626744,0.553156
max,0.675581,0.524594,0.675581,0.569437


# SSMBA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/ssmba/base_offensive_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/ssmba/base_offensive_'+str(perc)+'_ssmba'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/ssmba/base_offensive_'+str(perc)+'_ssmba'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/ssmba/base_offensive_'+str(perc)+'_ssmba'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/ssmba/base_offensive_'+str(perc)+'_ssmba'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 6s - loss: 0.6372 - accuracy: 0.6637 - 6s/epoch - 184ms/step
Epoch 2/30
32/32 - 4s - loss: 0.5772 - accuracy: 0.6706 - 4s/epoch - 135ms/step
Epoch 3/30
32/32 - 2s - loss: 0.4436 - accuracy: 0.6706 - 2s/epoch - 78ms/step
Epoch 4/30
32/32 - 3s - loss: 0.3210 - accuracy: 0.8657 - 3s/epoch - 79ms/step
Epoch 5/30
32/32 - 2s - loss: 0.1697 - accuracy: 0.9706 - 2s/epoch - 77ms/step
Epoch 6/30
32/32 - 3s - loss: 0.0994 - accuracy: 0.9775 - 3s/epoch - 81ms/step
Epoch 7/30
32/32 - 5s - loss: 0.0756 - accuracy: 0.9833 - 5s/epoch - 144ms/step
Epoch 8/30
32/32 - 4s - loss: 0.0513 - accuracy: 0.9853 - 4s/epoch - 134ms/step
Epoch 9/30
32/32 - 2s - loss: 0.0322 - accuracy: 0.9922 - 2s/epoch - 78ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0392 - accuracy: 0.9882 - 2s/epoch - 77ms/step
Epoch 11/30
32/32 - 2s - loss: 0.0355 - accuracy: 0.9951 - 2s/epoch - 77ms/step
Epoch 12/30
32/32 - 2s - loss: 0.0357 - accuracy: 0.9922 - 2s/epoch - 77ms/step
Epoch 13/30
32/32 - 4s - loss: 0.0367 - accur

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 5s - loss: 0.6493 - accuracy: 0.6608 - 5s/epoch - 148ms/step
Epoch 2/30
32/32 - 4s - loss: 0.5804 - accuracy: 0.6706 - 4s/epoch - 136ms/step
Epoch 3/30
32/32 - 3s - loss: 0.4675 - accuracy: 0.6706 - 3s/epoch - 99ms/step
Epoch 4/30
32/32 - 2s - loss: 0.3377 - accuracy: 0.7520 - 2s/epoch - 78ms/step
Epoch 5/30
32/32 - 2s - loss: 0.2338 - accuracy: 0.8833 - 2s/epoch - 76ms/step
Epoch 6/30
32/32 - 2s - loss: 0.1941 - accuracy: 0.9118 - 2s/epoch - 76ms/step
Epoch 7/30
32/32 - 3s - loss: 0.1904 - accuracy: 0.9049 - 3s/epoch - 108ms/step
Epoch 8/30
32/32 - 4s - loss: 0.1729 - accuracy: 0.9176 - 4s/epoch - 136ms/step
Epoch 9/30
32/32 - 3s - loss: 0.1828 - accuracy: 0.9039 - 3s/epoch - 108ms/step
Epoch 10/30
32/32 - 2s - loss: 0.1654 - accuracy: 0.9049 - 2s/epoch - 78ms/step
Epoch 11/30
32/32 - 2s - loss: 0.1392 - accuracy: 0.9069 - 2s/epoch - 77ms/step
Epoch 12/30
32/32 - 3s - loss: 0.1172 - accuracy: 0.9088 - 3s/epoch - 79ms/step
Epoch 13/30
32/32 - 3s - loss: 0.1196 - accu

...

Epoch 1/30
32/32 - 4s - loss: 0.6429 - accuracy: 0.6676 - 4s/epoch - 110ms/step
Epoch 2/30
32/32 - 3s - loss: 0.5990 - accuracy: 0.6706 - 3s/epoch - 83ms/step
Epoch 3/30
32/32 - 4s - loss: 0.4703 - accuracy: 0.7284 - 4s/epoch - 139ms/step
Epoch 4/30
32/32 - 4s - loss: 0.2231 - accuracy: 0.8686 - 4s/epoch - 132ms/step
Epoch 5/30
32/32 - 3s - loss: 0.1101 - accuracy: 0.9127 - 3s/epoch - 80ms/step
Epoch 6/30
32/32 - 2s - loss: 0.0858 - accuracy: 0.9686 - 2s/epoch - 77ms/step
Epoch 7/30
32/32 - 2s - loss: 0.0725 - accuracy: 0.9922 - 2s/epoch - 77ms/step
Epoch 8/30
32/32 - 2s - loss: 0.0674 - accuracy: 0.9902 - 2s/epoch - 77ms/step
Epoch 9/30
32/32 - 4s - loss: 0.0610 - accuracy: 0.9882 - 4s/epoch - 134ms/step
Epoch 10/30
32/32 - 4s - loss: 0.0656 - accuracy: 0.9951 - 4s/epoch - 137ms/step
Epoch 11/30
32/32 - 3s - loss: 0.0545 - accuracy: 0.9922 - 3s/epoch - 83ms/step
Epoch 12/30
32/32 - 2s - loss: 0.0541 - accuracy: 0.9951 - 2s/epoch - 78ms/step
Epoch 13/30
32/32 - 2s - loss: 0.0522 - accu

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 6s - loss: 0.6551 - accuracy: 0.6588 - 6s/epoch - 177ms/step
Epoch 2/30
32/32 - 4s - loss: 0.5864 - accuracy: 0.6745 - 4s/epoch - 133ms/step
Epoch 3/30
32/32 - 2s - loss: 0.4258 - accuracy: 0.7775 - 2s/epoch - 77ms/step
Epoch 4/30
32/32 - 2s - loss: 0.2223 - accuracy: 0.8931 - 2s/epoch - 77ms/step
Epoch 5/30
32/32 - 2s - loss: 0.1428 - accuracy: 0.9167 - 2s/epoch - 76ms/step
Epoch 6/30
32/32 - 2s - loss: 0.1311 - accuracy: 0.9088 - 2s/epoch - 78ms/step
Epoch 7/30
32/32 - 4s - loss: 0.1390 - accuracy: 0.9029 - 4s/epoch - 138ms/step
Epoch 8/30
32/32 - 4s - loss: 0.1371 - accuracy: 0.8980 - 4s/epoch - 134ms/step
Epoch 9/30
32/32 - 3s - loss: 0.1063 - accuracy: 0.9225 - 3s/epoch - 83ms/step
Epoch 10/30
32/32 - 2s - loss: 0.1110 - accuracy: 0.9245 - 2s/epoch - 78ms/step
Epoch 11/30
32/32 - 2s - loss: 0.1106 - accuracy: 0.9157 - 2s/epoch - 78ms/step
Epoch 12/30
32/32 - 3s - loss: 0.1210 - accuracy: 0.9157 - 3s/epoch - 79ms/step
Epoch 13/30
32/32 - 4s - loss: 0.1132 - accur

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 4s - loss: 0.6568 - accuracy: 0.6637 - 4s/epoch - 135ms/step
Epoch 2/30
32/32 - 4s - loss: 0.5885 - accuracy: 0.6706 - 4s/epoch - 135ms/step
Epoch 3/30
32/32 - 4s - loss: 0.4525 - accuracy: 0.6706 - 4s/epoch - 115ms/step
Epoch 4/30
32/32 - 2s - loss: 0.3142 - accuracy: 0.7706 - 2s/epoch - 77ms/step
Epoch 5/30
32/32 - 2s - loss: 0.2631 - accuracy: 0.9480 - 2s/epoch - 76ms/step
Epoch 6/30
32/32 - 2s - loss: 0.2330 - accuracy: 0.9755 - 2s/epoch - 76ms/step
Epoch 7/30
32/32 - 3s - loss: 0.2145 - accuracy: 0.9804 - 3s/epoch - 92ms/step
Epoch 8/30
32/32 - 4s - loss: 0.2016 - accuracy: 0.9833 - 4s/epoch - 137ms/step
Epoch 9/30
32/32 - 4s - loss: 0.1861 - accuracy: 0.9922 - 4s/epoch - 123ms/step
Epoch 10/30
32/32 - 2s - loss: 0.1832 - accuracy: 0.9853 - 2s/epoch - 77ms/step
Epoch 11/30
32/32 - 2s - loss: 0.1693 - accuracy: 0.9912 - 2s/epoch - 76ms/step
Epoch 12/30
32/32 - 2s - loss: 0.1656 - accuracy: 0.9873 - 2s/epoch - 76ms/step
Epoch 13/30
32/32 - 3s - loss: 0.1608 - accu

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.630233,0.536949,0.605814,0.555633
1,eda2,0.574419,0.523956,0.709302,0.545785
2,eda3,0.615116,0.506818,0.686047,0.580090
3,eda4,0.619767,0.530498,0.648837,0.549398
4,eda5,0.620930,0.528965,0.668605,0.554667


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.612093,0.525437,0.663721,0.557115
std,0.021763,0.011395,0.039290,0.013453
min,0.574419,0.506818,0.605814,0.545785
25%,0.615116,0.523956,0.648837,0.549398
50%,0.619767,0.528965,0.668605,0.554667
75%,0.620930,0.530498,0.686047,0.555633
max,0.630233,0.536949,0.709302,0.580090


# Checklist

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/checklist/base_offensive_'+str(perc)+'_checklist'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/checklist/base_offensive_'+str(perc)+'_checklist'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/checklist/base_offensive_'+str(perc)+'_checklist'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/checklist/base_offensive_'+str(perc)+'_checklist'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/checklist/base_offensive_'+str(perc)+'_checklist'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
20/20 - 4s - loss: 0.6554 - accuracy: 0.6498 - 4s/epoch - 192ms/step
Epoch 2/30
20/20 - 1s - loss: 0.5404 - accuracy: 0.6661 - 1s/epoch - 72ms/step
Epoch 3/30
20/20 - 1s - loss: 0.4339 - accuracy: 0.6661 - 1s/epoch - 72ms/step
Epoch 4/30
20/20 - 1s - loss: 0.3579 - accuracy: 0.7834 - 1s/epoch - 71ms/step
Epoch 5/30
20/20 - 1s - loss: 0.2957 - accuracy: 0.9349 - 1s/epoch - 71ms/step
Epoch 6/30
20/20 - 1s - loss: 0.2621 - accuracy: 0.9691 - 1s/epoch - 71ms/step
Epoch 7/30
20/20 - 1s - loss: 0.2323 - accuracy: 0.9853 - 1s/epoch - 72ms/step
Epoch 8/30
20/20 - 2s - loss: 0.1712 - accuracy: 0.9870 - 2s/epoch - 88ms/step
Epoch 9/30
20/20 - 3s - loss: 0.1263 - accuracy: 0.9935 - 3s/epoch - 131ms/step
Epoch 10/30
20/20 - 3s - loss: 0.1281 - accuracy: 0.9919 - 3s/epoch - 126ms/step
Epoch 11/30
20/20 - 2s - loss: 0.1161 - accuracy: 0.9919 - 2s/epoch - 123ms/step
Epoch 12/30
20/20 - 2s - loss: 0.0983 - accuracy: 0.9951 - 2s/epoch - 86ms/step
Epoch 13/30
20/20 - 1s - loss: 0.1056 - accur

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
19/19 - 2s - loss: 0.6301 - accuracy: 0.6723 - 2s/epoch - 128ms/step
Epoch 2/30
19/19 - 1s - loss: 0.5220 - accuracy: 0.6757 - 1s/epoch - 73ms/step
Epoch 3/30
19/19 - 2s - loss: 0.3760 - accuracy: 0.7725 - 2s/epoch - 93ms/step
Epoch 4/30
19/19 - 2s - loss: 0.2494 - accuracy: 0.8846 - 2s/epoch - 131ms/step
Epoch 5/30
19/19 - 2s - loss: 0.1677 - accuracy: 0.9457 - 2s/epoch - 130ms/step
Epoch 6/30
19/19 - 2s - loss: 0.1424 - accuracy: 0.9711 - 2s/epoch - 127ms/step
Epoch 7/30
19/19 - 2s - loss: 0.1335 - accuracy: 0.9864 - 2s/epoch - 93ms/step
Epoch 8/30
19/19 - 1s - loss: 0.1165 - accuracy: 0.9847 - 1s/epoch - 74ms/step
Epoch 9/30
19/19 - 1s - loss: 0.1166 - accuracy: 0.9830 - 1s/epoch - 73ms/step
Epoch 10/30
19/19 - 1s - loss: 0.1060 - accuracy: 0.9966 - 1s/epoch - 74ms/step
Epoch 11/30
19/19 - 1s - loss: 0.1226 - accuracy: 0.9881 - 1s/epoch - 74ms/step
Epoch 12/30
19/19 - 1s - loss: 0.0897 - accuracy: 0.9983 - 1s/epoch - 73ms/step
Epoch 13/30
19/19 - 1s - loss: 0.0928 - accur

...

Epoch 1/30
19/19 - 4s - loss: 0.6515 - accuracy: 0.6533 - 4s/epoch - 218ms/step
Epoch 2/30
19/19 - 2s - loss: 0.5582 - accuracy: 0.6801 - 2s/epoch - 130ms/step
Epoch 3/30
19/19 - 2s - loss: 0.4574 - accuracy: 0.7052 - 2s/epoch - 89ms/step
Epoch 4/30
19/19 - 1s - loss: 0.3503 - accuracy: 0.7822 - 1s/epoch - 76ms/step
Epoch 5/30
19/19 - 1s - loss: 0.2347 - accuracy: 0.9045 - 1s/epoch - 75ms/step
Epoch 6/30
19/19 - 1s - loss: 0.1739 - accuracy: 0.9380 - 1s/epoch - 74ms/step
Epoch 7/30
19/19 - 1s - loss: 0.1144 - accuracy: 0.9665 - 1s/epoch - 73ms/step
Epoch 8/30
19/19 - 1s - loss: 0.1092 - accuracy: 0.9665 - 1s/epoch - 73ms/step
Epoch 9/30
19/19 - 1s - loss: 0.0691 - accuracy: 0.9698 - 1s/epoch - 74ms/step
Epoch 10/30
19/19 - 2s - loss: 0.0856 - accuracy: 0.9732 - 2s/epoch - 112ms/step
Epoch 11/30
19/19 - 3s - loss: 0.0808 - accuracy: 0.9749 - 3s/epoch - 132ms/step
Epoch 12/30
19/19 - 2s - loss: 0.0761 - accuracy: 0.9883 - 2s/epoch - 126ms/step
Epoch 13/30
19/19 - 2s - loss: 0.0777 - accu

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
20/20 - 2s - loss: 0.6511 - accuracy: 0.6410 - 2s/epoch - 125ms/step
Epoch 2/30
20/20 - 1s - loss: 0.5839 - accuracy: 0.6525 - 1s/epoch - 71ms/step
Epoch 3/30
20/20 - 1s - loss: 0.4962 - accuracy: 0.6541 - 1s/epoch - 73ms/step
Epoch 4/30
20/20 - 1s - loss: 0.3882 - accuracy: 0.7475 - 1s/epoch - 71ms/step
Epoch 5/30
20/20 - 1s - loss: 0.2428 - accuracy: 0.8836 - 1s/epoch - 72ms/step
Epoch 6/30
20/20 - 1s - loss: 0.1513 - accuracy: 0.9459 - 1s/epoch - 71ms/step
Epoch 7/30
20/20 - 2s - loss: 0.1010 - accuracy: 0.9656 - 2s/epoch - 113ms/step
Epoch 8/30
20/20 - 3s - loss: 0.0804 - accuracy: 0.9689 - 3s/epoch - 131ms/step
Epoch 9/30
20/20 - 3s - loss: 0.0662 - accuracy: 0.9787 - 3s/epoch - 126ms/step
Epoch 10/30
20/20 - 2s - loss: 0.0814 - accuracy: 0.9721 - 2s/epoch - 120ms/step
Epoch 11/30
20/20 - 1s - loss: 0.0819 - accuracy: 0.9721 - 1s/epoch - 73ms/step
Epoch 12/30
20/20 - 1s - loss: 0.0747 - accuracy: 0.9803 - 1s/epoch - 71ms/step
Epoch 13/30
20/20 - 1s - loss: 0.0604 - accu

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
19/19 - 4s - loss: 0.6397 - accuracy: 0.6750 - 4s/epoch - 223ms/step
Epoch 2/30
19/19 - 2s - loss: 0.5586 - accuracy: 0.6900 - 2s/epoch - 128ms/step
Epoch 3/30
19/19 - 2s - loss: 0.4509 - accuracy: 0.6900 - 2s/epoch - 119ms/step
Epoch 4/30
19/19 - 1s - loss: 0.3269 - accuracy: 0.7500 - 1s/epoch - 73ms/step
Epoch 5/30
19/19 - 1s - loss: 0.1998 - accuracy: 0.9367 - 1s/epoch - 72ms/step
Epoch 6/30
19/19 - 1s - loss: 0.1569 - accuracy: 0.9700 - 1s/epoch - 73ms/step
Epoch 7/30
19/19 - 1s - loss: 0.1243 - accuracy: 0.9850 - 1s/epoch - 73ms/step
Epoch 8/30
19/19 - 1s - loss: 0.1126 - accuracy: 0.9883 - 1s/epoch - 73ms/step
Epoch 9/30
19/19 - 1s - loss: 0.1133 - accuracy: 0.9850 - 1s/epoch - 73ms/step
Epoch 10/30
19/19 - 1s - loss: 0.1144 - accuracy: 0.9900 - 1s/epoch - 72ms/step
Epoch 11/30
19/19 - 2s - loss: 0.0998 - accuracy: 0.9933 - 2s/epoch - 125ms/step
Epoch 12/30
19/19 - 3s - loss: 0.0903 - accuracy: 0.9933 - 3s/epoch - 132ms/step
Epoch 13/30
19/19 - 2s - loss: 0.1012 - accu

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.710465,0.467365,0.668605,0.556068
1,eda2,0.693023,0.452520,0.623256,0.542129
2,eda3,0.662791,0.488511,0.612791,0.567244
3,eda4,0.643023,0.518766,0.504651,0.488241
4,eda5,0.673256,0.509410,0.682558,0.587404


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.676512,0.487314,0.618372,0.548217
std,0.026200,0.027783,0.070055,0.037395
min,0.643023,0.452520,0.504651,0.488241
25%,0.662791,0.467365,0.612791,0.542129
50%,0.673256,0.488511,0.623256,0.556068
75%,0.693023,0.509410,0.668605,0.567244
max,0.710465,0.518766,0.682558,0.587404


# Back Translation

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/back_translation/base_offensive_'+str(perc)+'_back_translation'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/back_translation/base_offensive_'+str(perc)+'_back_translation'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/back_translation/base_offensive_'+str(perc)+'_back_translation'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/back_translation/base_offensive_'+str(perc)+'_back_translation'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/back_translation/base_offensive_'+str(perc)+'_back_translation'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
11/11 - 2s - loss: 0.6748 - accuracy: 0.6431 - 2s/epoch - 171ms/step
Epoch 2/30
11/11 - 1s - loss: 0.6051 - accuracy: 0.6696 - 806ms/epoch - 73ms/step
Epoch 3/30
11/11 - 1s - loss: 0.5735 - accuracy: 0.6726 - 812ms/epoch - 74ms/step
Epoch 4/30
11/11 - 1s - loss: 0.5253 - accuracy: 0.7227 - 801ms/epoch - 73ms/step
Epoch 5/30
11/11 - 1s - loss: 0.4741 - accuracy: 0.7316 - 809ms/epoch - 74ms/step
Epoch 6/30
11/11 - 1s - loss: 0.3967 - accuracy: 0.7522 - 795ms/epoch - 72ms/step
Epoch 7/30
11/11 - 1s - loss: 0.3138 - accuracy: 0.8053 - 813ms/epoch - 74ms/step
Epoch 8/30
11/11 - 1s - loss: 0.2627 - accuracy: 0.8496 - 803ms/epoch - 73ms/step
Epoch 9/30
11/11 - 1s - loss: 0.2186 - accuracy: 0.9322 - 1s/epoch - 132ms/step
Epoch 10/30
11/11 - 1s - loss: 0.1828 - accuracy: 0.9558 - 1s/epoch - 130ms/step
Epoch 11/30
11/11 - 1s - loss: 0.1748 - accuracy: 0.9646 - 1s/epoch - 131ms/step
Epoch 12/30
11/11 - 1s - loss: 0.1606 - accuracy: 0.9617 - 1s/epoch - 123ms/step
Epoch 13/30
11/11 - 1s 

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
11/11 - 2s - loss: 0.6642 - accuracy: 0.6283 - 2s/epoch - 168ms/step
Epoch 2/30
11/11 - 1s - loss: 0.6161 - accuracy: 0.6726 - 792ms/epoch - 72ms/step
Epoch 3/30
11/11 - 1s - loss: 0.5936 - accuracy: 0.6726 - 812ms/epoch - 74ms/step
Epoch 4/30
11/11 - 1s - loss: 0.5549 - accuracy: 0.6726 - 800ms/epoch - 73ms/step
Epoch 5/30
11/11 - 1s - loss: 0.5179 - accuracy: 0.6726 - 822ms/epoch - 75ms/step
Epoch 6/30
11/11 - 1s - loss: 0.4873 - accuracy: 0.6726 - 1s/epoch - 94ms/step
Epoch 7/30
11/11 - 1s - loss: 0.4592 - accuracy: 0.6726 - 1s/epoch - 132ms/step
Epoch 8/30
11/11 - 1s - loss: 0.4176 - accuracy: 0.6726 - 1s/epoch - 131ms/step
Epoch 9/30
11/11 - 1s - loss: 0.3934 - accuracy: 0.6726 - 1s/epoch - 132ms/step
Epoch 10/30
11/11 - 1s - loss: 0.3756 - accuracy: 0.6726 - 1s/epoch - 124ms/step
Epoch 11/30
11/11 - 1s - loss: 0.3379 - accuracy: 0.7670 - 1s/epoch - 124ms/step
Epoch 12/30
11/11 - 1s - loss: 0.3218 - accuracy: 0.8791 - 1s/epoch - 122ms/step
Epoch 13/30
11/11 - 1s - loss:

...

Epoch 1/30
11/11 - 2s - loss: 0.6700 - accuracy: 0.6235 - 2s/epoch - 168ms/step
Epoch 2/30
11/11 - 1s - loss: 0.6347 - accuracy: 0.6706 - 813ms/epoch - 74ms/step
Epoch 3/30
11/11 - 1s - loss: 0.6140 - accuracy: 0.6706 - 800ms/epoch - 73ms/step
Epoch 4/30
11/11 - 1s - loss: 0.5592 - accuracy: 0.6706 - 800ms/epoch - 73ms/step
Epoch 5/30
11/11 - 1s - loss: 0.5393 - accuracy: 0.6706 - 811ms/epoch - 74ms/step
Epoch 6/30
11/11 - 1s - loss: 0.4921 - accuracy: 0.6706 - 797ms/epoch - 72ms/step
Epoch 7/30
11/11 - 1s - loss: 0.4609 - accuracy: 0.6706 - 822ms/epoch - 75ms/step
Epoch 8/30
11/11 - 1s - loss: 0.4037 - accuracy: 0.6706 - 1s/epoch - 97ms/step
Epoch 9/30
11/11 - 1s - loss: 0.3557 - accuracy: 0.6706 - 1s/epoch - 134ms/step
Epoch 10/30
11/11 - 1s - loss: 0.3355 - accuracy: 0.7882 - 1s/epoch - 135ms/step
Epoch 11/30
11/11 - 1s - loss: 0.2989 - accuracy: 0.9088 - 1s/epoch - 126ms/step
Epoch 12/30
11/11 - 1s - loss: 0.2798 - accuracy: 0.9265 - 1s/epoch - 127ms/step
Epoch 13/30
11/11 - 1s - l

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


11/11 - 2s - loss: 0.6797 - accuracy: 0.6283 - 2s/epoch - 170ms/step
Epoch 2/30
11/11 - 1s - loss: 0.6430 - accuracy: 0.6696 - 803ms/epoch - 73ms/step
Epoch 3/30
11/11 - 1s - loss: 0.6160 - accuracy: 0.6696 - 812ms/epoch - 74ms/step
Epoch 4/30
11/11 - 1s - loss: 0.5958 - accuracy: 0.6667 - 799ms/epoch - 73ms/step
Epoch 5/30
11/11 - 1s - loss: 0.5565 - accuracy: 0.6873 - 815ms/epoch - 74ms/step
Epoch 6/30
11/11 - 1s - loss: 0.5063 - accuracy: 0.7168 - 785ms/epoch - 71ms/step
Epoch 7/30
11/11 - 1s - loss: 0.4357 - accuracy: 0.7935 - 799ms/epoch - 73ms/step
Epoch 8/30
11/11 - 1s - loss: 0.3677 - accuracy: 0.7965 - 810ms/epoch - 74ms/step
Epoch 9/30
11/11 - 1s - loss: 0.3005 - accuracy: 0.7994 - 825ms/epoch - 75ms/step
Epoch 10/30
11/11 - 1s - loss: 0.2546 - accuracy: 0.8496 - 808ms/epoch - 73ms/step
Epoch 11/30
11/11 - 1s - loss: 0.2451 - accuracy: 0.8083 - 1s/epoch - 125ms/step
Epoch 12/30
11/11 - 1s - loss: 0.2469 - accuracy: 0.8142 - 1s/epoch - 128ms/step
Epoch 13/30
11/11 - 1s - loss:

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


11/11 - 2s - loss: 0.6556 - accuracy: 0.6618 - 2s/epoch - 169ms/step
Epoch 2/30
11/11 - 1s - loss: 0.6386 - accuracy: 0.6706 - 833ms/epoch - 76ms/step
Epoch 3/30
11/11 - 1s - loss: 0.6099 - accuracy: 0.6706 - 825ms/epoch - 75ms/step
Epoch 4/30
11/11 - 1s - loss: 0.5754 - accuracy: 0.6706 - 812ms/epoch - 74ms/step
Epoch 5/30
11/11 - 1s - loss: 0.5387 - accuracy: 0.6706 - 826ms/epoch - 75ms/step
Epoch 6/30
11/11 - 1s - loss: 0.4973 - accuracy: 0.6706 - 1s/epoch - 110ms/step
Epoch 7/30
11/11 - 2s - loss: 0.4588 - accuracy: 0.6706 - 2s/epoch - 138ms/step
Epoch 8/30
11/11 - 1s - loss: 0.4117 - accuracy: 0.6706 - 1s/epoch - 130ms/step
Epoch 9/30
11/11 - 1s - loss: 0.3651 - accuracy: 0.6706 - 1s/epoch - 128ms/step
Epoch 10/30
11/11 - 1s - loss: 0.3289 - accuracy: 0.8059 - 1s/epoch - 128ms/step
Epoch 11/30
11/11 - 1s - loss: 0.3000 - accuracy: 0.9412 - 1s/epoch - 130ms/step
Epoch 12/30
11/11 - 1s - loss: 0.2796 - accuracy: 0.9265 - 1s/epoch - 115ms/step
Epoch 13/30
11/11 - 1s - loss: 0.2665 - 

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.624419,0.495290,0.582558,0.512005
1,eda2,0.626744,0.516260,0.619767,0.533971
2,eda3,0.706977,0.465570,0.613953,0.556301
3,eda4,0.712791,0.512797,0.688372,0.577875
4,eda5,0.720930,0.469594,0.655814,0.575538


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.678372,0.491902,0.632093,0.551138
std,0.048452,0.023624,0.040819,0.028120
min,0.624419,0.465570,0.582558,0.512005
25%,0.626744,0.469594,0.613953,0.533971
50%,0.706977,0.495290,0.619767,0.556301
75%,0.712791,0.512797,0.655814,0.575538
max,0.720930,0.516260,0.688372,0.577875


# LAMBADA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 4s - loss: 0.3220 - accuracy: 0.9324 - 4s/epoch - 118ms/step
Epoch 2/30
32/32 - 2s - loss: 0.2258 - accuracy: 0.9451 - 2s/epoch - 77ms/step
Epoch 3/30
32/32 - 2s - loss: 0.1952 - accuracy: 0.9451 - 2s/epoch - 76ms/step
Epoch 4/30
32/32 - 3s - loss: 0.1760 - accuracy: 0.9451 - 3s/epoch - 83ms/step
Epoch 5/30
32/32 - 4s - loss: 0.1438 - accuracy: 0.9451 - 4s/epoch - 135ms/step
Epoch 6/30
32/32 - 4s - loss: 0.1105 - accuracy: 0.9451 - 4s/epoch - 129ms/step
Epoch 7/30
32/32 - 2s - loss: 0.1005 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 8/30
32/32 - 2s - loss: 0.0889 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 9/30
32/32 - 2s - loss: 0.0806 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0768 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 11/30
32/32 - 4s - loss: 0.0733 - accuracy: 0.9451 - 4s/epoch - 121ms/step
Epoch 12/30
32/32 - 4s - loss: 0.0644 - accuracy: 0.9451 - 4s/epoch - 130ms/step
Epoch 13/30
32/32 - 3s - loss: 0.0683 - accu

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 6s - loss: 0.3076 - accuracy: 0.9382 - 6s/epoch - 178ms/step
Epoch 2/30
32/32 - 4s - loss: 0.2462 - accuracy: 0.9451 - 4s/epoch - 116ms/step
Epoch 3/30
32/32 - 2s - loss: 0.2214 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 4/30
32/32 - 2s - loss: 0.1910 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 5/30
32/32 - 2s - loss: 0.1423 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 6/30
32/32 - 2s - loss: 0.1058 - accuracy: 0.9451 - 2s/epoch - 76ms/step
Epoch 7/30
32/32 - 4s - loss: 0.0978 - accuracy: 0.9451 - 4s/epoch - 134ms/step
Epoch 8/30
32/32 - 4s - loss: 0.0815 - accuracy: 0.9451 - 4s/epoch - 127ms/step
Epoch 9/30
32/32 - 3s - loss: 0.0707 - accuracy: 0.9451 - 3s/epoch - 80ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0736 - accuracy: 0.9451 - 2s/epoch - 73ms/step
Epoch 11/30
32/32 - 2s - loss: 0.0661 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 12/30
32/32 - 2s - loss: 0.0631 - accuracy: 0.9451 - 2s/epoch - 73ms/step
Epoch 13/30
32/32 - 4s - loss: 0.0690 - accur

...

Epoch 1/30
32/32 - 3s - loss: 0.3178 - accuracy: 0.9402 - 3s/epoch - 108ms/step
Epoch 2/30
32/32 - 2s - loss: 0.2543 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 3/30
32/32 - 3s - loss: 0.2327 - accuracy: 0.9451 - 3s/epoch - 105ms/step
Epoch 4/30
32/32 - 4s - loss: 0.1902 - accuracy: 0.9451 - 4s/epoch - 133ms/step
Epoch 5/30
32/32 - 3s - loss: 0.1617 - accuracy: 0.9451 - 3s/epoch - 109ms/step
Epoch 6/30
32/32 - 2s - loss: 0.1182 - accuracy: 0.9461 - 2s/epoch - 75ms/step
Epoch 7/30
32/32 - 2s - loss: 0.1013 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 8/30
32/32 - 2s - loss: 0.0805 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 9/30
32/32 - 3s - loss: 0.0731 - accuracy: 0.9451 - 3s/epoch - 83ms/step
Epoch 10/30
32/32 - 4s - loss: 0.0607 - accuracy: 0.9451 - 4s/epoch - 135ms/step
Epoch 11/30
32/32 - 4s - loss: 0.0735 - accuracy: 0.9451 - 4s/epoch - 125ms/step
Epoch 12/30
32/32 - 2s - loss: 0.0655 - accuracy: 0.9451 - 2s/epoch - 77ms/step
Epoch 13/30
32/32 - 2s - loss: 0.0652 - acc

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 6s - loss: 0.3099 - accuracy: 0.9235 - 6s/epoch - 175ms/step
Epoch 2/30
32/32 - 2s - loss: 0.2526 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 3/30
32/32 - 2s - loss: 0.2148 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 4/30
32/32 - 2s - loss: 0.1941 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 5/30
32/32 - 2s - loss: 0.1709 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 6/30
32/32 - 4s - loss: 0.1280 - accuracy: 0.9451 - 4s/epoch - 124ms/step
Epoch 7/30
32/32 - 4s - loss: 0.0965 - accuracy: 0.9451 - 4s/epoch - 131ms/step
Epoch 8/30
32/32 - 3s - loss: 0.0916 - accuracy: 0.9451 - 3s/epoch - 89ms/step
Epoch 9/30
32/32 - 2s - loss: 0.0821 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0726 - accuracy: 0.9451 - 2s/epoch - 76ms/step
Epoch 11/30
32/32 - 2s - loss: 0.0772 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 12/30
32/32 - 4s - loss: 0.0620 - accuracy: 0.9451 - 4s/epoch - 110ms/step
Epoch 13/30
32/32 - 4s - loss: 0.0687 - accur

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 6s - loss: 0.3312 - accuracy: 0.9216 - 6s/epoch - 177ms/step
Epoch 2/30
Epoch 3/30
32/32 - 2s - loss: 0.2620 - accuracy: 0.9451 - 2s/epoch - 73ms/step
Epoch 4/30
32/32 - 2s - loss: 0.2544 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 5/30
32/32 - 2s - loss: 0.2201 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 6/30
32/32 - 4s - loss: 0.2029 - accuracy: 0.9451 - 4s/epoch - 122ms/step
Epoch 7/30
32/32 - 4s - loss: 0.1763 - accuracy: 0.9451 - 4s/epoch - 130ms/step
Epoch 8/30
32/32 - 3s - loss: 0.1598 - accuracy: 0.9451 - 3s/epoch - 95ms/step
Epoch 9/30
32/32 - 2s - loss: 0.1303 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 10/30
32/32 - 2s - loss: 0.1313 - accuracy: 0.9451 - 2s/epoch - 75ms/step
Epoch 11/30
32/32 - 2s - loss: 0.1184 - accuracy: 0.9451 - 2s/epoch - 74ms/step
Epoch 12/30
32/32 - 3s - loss: 0.1237 - accuracy: 0.9451 - 3s/epoch - 102ms/step
Epoch 13/30
32/32 - 4s - loss: 0.1164 - accuracy: 0.9539 - 4s/epoch - 133ms/step
Epoch 14/30
32/32 - 4s - loss: 0

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.612791,0.471209,0.660465,0.469477
1,eda2,0.619767,0.471962,0.655814,0.455039
2,eda3,0.618605,0.461992,0.680233,0.468242
3,eda4,0.648837,0.477988,0.681395,0.486287
4,eda5,0.617442,0.496869,0.548837,0.473129


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.623488,0.476004,0.645349,0.470435
std,0.014416,0.012990,0.055156,0.011195
min,0.612791,0.461992,0.548837,0.455039
25%,0.617442,0.471209,0.655814,0.468242
50%,0.618605,0.471962,0.660465,0.469477
75%,0.619767,0.477988,0.680233,0.473129
max,0.648837,0.496869,0.681395,0.486287


# Contrastive LAMBADA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada_contrastive'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada_contrastive'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada_contrastive'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada_contrastive'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/lambada/base_offensive_'+str(perc)+'_lambada_contrastive'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 3s - loss: 0.5415 - accuracy: 0.7912 - 3s/epoch - 109ms/step
Epoch 2/30
32/32 - 3s - loss: 0.4618 - accuracy: 0.7990 - 3s/epoch - 87ms/step
Epoch 3/30
32/32 - 4s - loss: 0.4019 - accuracy: 0.7990 - 4s/epoch - 133ms/step
Epoch 4/30
32/32 - 4s - loss: 0.3181 - accuracy: 0.8147 - 4s/epoch - 125ms/step
Epoch 5/30
32/32 - 2s - loss: 0.2278 - accuracy: 0.8794 - 2s/epoch - 75ms/step
Epoch 6/30
32/32 - 2s - loss: 0.1471 - accuracy: 0.9216 - 2s/epoch - 73ms/step
Epoch 7/30
32/32 - 2s - loss: 0.1043 - accuracy: 0.9304 - 2s/epoch - 75ms/step
Epoch 8/30
32/32 - 2s - loss: 0.0966 - accuracy: 0.9382 - 2s/epoch - 74ms/step
Epoch 9/30
32/32 - 4s - loss: 0.0789 - accuracy: 0.9402 - 4s/epoch - 125ms/step
Epoch 10/30
32/32 - 4s - loss: 0.0750 - accuracy: 0.9627 - 4s/epoch - 130ms/step
Epoch 11/30
32/32 - 3s - loss: 0.0772 - accuracy: 0.9578 - 3s/epoch - 92ms/step
Epoch 12/30
32/32 - 2s - loss: 0.0552 - accuracy: 0.9784 - 2s/epoch - 73ms/step
Epoch 13/30
32/32 - 2s - loss: 0.0587 - accu

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 4s - loss: 0.6260 - accuracy: 0.7029 - 4s/epoch - 111ms/step
Epoch 2/30
32/32 - 2s - loss: 0.5270 - accuracy: 0.7098 - 2s/epoch - 73ms/step
Epoch 3/30
32/32 - 2s - loss: 0.4527 - accuracy: 0.7510 - 2s/epoch - 73ms/step
Epoch 4/30
32/32 - 3s - loss: 0.3736 - accuracy: 0.7863 - 3s/epoch - 80ms/step
Epoch 5/30
32/32 - 4s - loss: 0.3082 - accuracy: 0.8480 - 4s/epoch - 130ms/step
Epoch 6/30
32/32 - 4s - loss: 0.2015 - accuracy: 0.9196 - 4s/epoch - 127ms/step
Epoch 7/30
32/32 - 2s - loss: 0.1483 - accuracy: 0.9431 - 2s/epoch - 78ms/step
Epoch 8/30
32/32 - 2s - loss: 0.1220 - accuracy: 0.9529 - 2s/epoch - 74ms/step
Epoch 9/30
32/32 - 2s - loss: 0.0853 - accuracy: 0.9755 - 2s/epoch - 74ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0884 - accuracy: 0.9676 - 2s/epoch - 74ms/step
Epoch 11/30
32/32 - 4s - loss: 0.0767 - accuracy: 0.9765 - 4s/epoch - 116ms/step
Epoch 12/30
32/32 - 4s - loss: 0.0687 - accuracy: 0.9775 - 4s/epoch - 130ms/step
Epoch 13/30
32/32 - 3s - loss: 0.0547 - accu

...

Epoch 1/30
32/32 - 4s - loss: 0.6289 - accuracy: 0.6961 - 4s/epoch - 122ms/step
Epoch 2/30
32/32 - 2s - loss: 0.6202 - accuracy: 0.7088 - 2s/epoch - 75ms/step
Epoch 3/30
32/32 - 2s - loss: 0.5715 - accuracy: 0.7088 - 2s/epoch - 75ms/step
Epoch 4/30
32/32 - 2s - loss: 0.5230 - accuracy: 0.7088 - 2s/epoch - 74ms/step
Epoch 5/30
32/32 - 4s - loss: 0.4515 - accuracy: 0.7088 - 4s/epoch - 129ms/step
Epoch 6/30
32/32 - 4s - loss: 0.4078 - accuracy: 0.7088 - 4s/epoch - 127ms/step
Epoch 7/30
32/32 - 3s - loss: 0.3779 - accuracy: 0.7088 - 3s/epoch - 88ms/step
Epoch 8/30
32/32 - 2s - loss: 0.3701 - accuracy: 0.7088 - 2s/epoch - 74ms/step
Epoch 9/30
32/32 - 2s - loss: 0.3695 - accuracy: 0.7088 - 2s/epoch - 75ms/step
Epoch 10/30
32/32 - 2s - loss: 0.3547 - accuracy: 0.7108 - 2s/epoch - 73ms/step
Epoch 11/30
32/32 - 3s - loss: 0.3431 - accuracy: 0.8039 - 3s/epoch - 106ms/step
Epoch 12/30
32/32 - 4s - loss: 0.3373 - accuracy: 0.8069 - 4s/epoch - 131ms/step
Epoch 13/30
32/32 - 4s - loss: 0.3265 - accu

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 4s - loss: 0.6839 - accuracy: 0.5725 - 4s/epoch - 122ms/step
Epoch 2/30
32/32 - 2s - loss: 0.6474 - accuracy: 0.6108 - 2s/epoch - 74ms/step
Epoch 3/30
32/32 - 2s - loss: 0.5745 - accuracy: 0.6500 - 2s/epoch - 75ms/step
Epoch 4/30
32/32 - 2s - loss: 0.4853 - accuracy: 0.7608 - 2s/epoch - 74ms/step
Epoch 5/30
32/32 - 4s - loss: 0.3457 - accuracy: 0.8618 - 4s/epoch - 127ms/step
Epoch 6/30
32/32 - 4s - loss: 0.2353 - accuracy: 0.9020 - 4s/epoch - 129ms/step
Epoch 7/30
32/32 - 3s - loss: 0.1907 - accuracy: 0.9412 - 3s/epoch - 89ms/step
Epoch 8/30
32/32 - 2s - loss: 0.1211 - accuracy: 0.9618 - 2s/epoch - 74ms/step
Epoch 9/30
32/32 - 2s - loss: 0.1206 - accuracy: 0.9569 - 2s/epoch - 74ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0988 - accuracy: 0.9784 - 2s/epoch - 74ms/step
Epoch 11/30
32/32 - 3s - loss: 0.0781 - accuracy: 0.9725 - 3s/epoch - 105ms/step
Epoch 12/30
32/32 - 4s - loss: 0.0432 - accuracy: 0.9922 - 4s/epoch - 130ms/step
Epoch 13/30
32/32 - 4s - loss: 0.0523 - accu

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 5s - loss: 0.6466 - accuracy: 0.6696 - 5s/epoch - 158ms/step
Epoch 2/30
32/32 - 2s - loss: 0.5857 - accuracy: 0.6765 - 2s/epoch - 74ms/step
Epoch 3/30
32/32 - 2s - loss: 0.5057 - accuracy: 0.6765 - 2s/epoch - 73ms/step
Epoch 4/30
32/32 - 2s - loss: 0.4447 - accuracy: 0.6784 - 2s/epoch - 74ms/step
Epoch 5/30
32/32 - 3s - loss: 0.3561 - accuracy: 0.8265 - 3s/epoch - 85ms/step
Epoch 6/30
32/32 - 4s - loss: 0.2388 - accuracy: 0.9137 - 4s/epoch - 134ms/step
Epoch 7/30
32/32 - 4s - loss: 0.2013 - accuracy: 0.9549 - 4s/epoch - 128ms/step
Epoch 8/30
32/32 - 2s - loss: 0.1815 - accuracy: 0.9598 - 2s/epoch - 75ms/step
Epoch 9/30
32/32 - 2s - loss: 0.1545 - accuracy: 0.9745 - 2s/epoch - 74ms/step
Epoch 10/30
32/32 - 2s - loss: 0.1441 - accuracy: 0.9686 - 2s/epoch - 74ms/step
Epoch 11/30
32/32 - 2s - loss: 0.1374 - accuracy: 0.9696 - 2s/epoch - 74ms/step
Epoch 12/30
32/32 - 4s - loss: 0.1154 - accuracy: 0.9833 - 4s/epoch - 123ms/step
Epoch 13/30
32/32 - 4s - loss: 0.1144 - accur

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.567442,0.523929,0.666279,0.567498
1,eda2,0.561628,0.522908,0.562791,0.522434
2,eda3,0.565116,0.534068,0.687209,0.628041
3,eda4,0.640698,0.552957,0.633721,0.548854
4,eda5,0.633721,0.554317,0.547674,0.509816


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.593721,0.537636,0.619535,0.555329
std,0.039830,0.015252,0.061947,0.046450
min,0.561628,0.522908,0.547674,0.509816
25%,0.565116,0.523929,0.562791,0.522434
50%,0.567442,0.534068,0.633721,0.548854
75%,0.633721,0.552957,0.666279,0.567498
max,0.640698,0.554317,0.687209,0.628041


# LLAMA 2 Paraphraser

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/gptparaphrase/base_offensive_'+str(perc)+'_gptparaphrase'+str(i+1)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/gptparaphrase/base_offensive_'+str(perc)+'_gptparaphrase'+str(i+2)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/gptparaphrase/base_offensive_'+str(perc)+'_gptparaphrase'+str(i+3)+'.pkl',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/gptparaphrase/base_offensive_'+str(perc)+'_gptparaphrase'+str(i+4)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/gptparaphrase/base_offensive_'+str(perc)+'_gptparaphrase'+str(i+5)+'.pkl',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal_pkl(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-8-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
31/31 - 5s - loss: 0.6539 - accuracy: 0.6613 - 5s/epoch - 176ms/step
Epoch 2/30
31/31 - 4s - loss: 0.6212 - accuracy: 0.6684 - 4s/epoch - 132ms/step
Epoch 3/30
31/31 - 3s - loss: 0.5545 - accuracy: 0.6684 - 3s/epoch - 107ms/step
Epoch 4/30
31/31 - 2s - loss: 0.4111 - accuracy: 0.7401 - 2s/epoch - 75ms/step
Epoch 5/30
31/31 - 2s - loss: 0.2643 - accuracy: 0.8028 - 2s/epoch - 76ms/step
Epoch 6/30
31/31 - 2s - loss: 0.2136 - accuracy: 0.8322 - 2s/epoch - 77ms/step
Epoch 7/30
31/31 - 3s - loss: 0.1725 - accuracy: 0.9181 - 3s/epoch - 89ms/step
Epoch 8/30
31/31 - 4s - loss: 0.1698 - accuracy: 0.9141 - 4s/epoch - 138ms/step
Epoch 9/30
31/31 - 4s - loss: 0.1794 - accuracy: 0.9171 - 4s/epoch - 128ms/step
Epoch 10/30
31/31 - 2s - loss: 0.1702 - accuracy: 0.9171 - 2s/epoch - 78ms/step
Epoch 11/30
31/31 - 2s - loss: 0.1847 - accuracy: 0.9019 - 2s/epoch - 76ms/step
Epoch 12/30
31/31 - 2s - loss: 0.1408 - accuracy: 0.9373 - 2s/epoch - 76ms/step
Epoch 13/30
31/31 - 2s - loss: 0.1262 - accu

<ipython-input-8-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
30/30 - 5s - loss: 0.6528 - accuracy: 0.6593 - 5s/epoch - 181ms/step
Epoch 2/30
30/30 - 4s - loss: 0.6185 - accuracy: 0.6677 - 4s/epoch - 127ms/step
Epoch 3/30
30/30 - 2s - loss: 0.5193 - accuracy: 0.6677 - 2s/epoch - 75ms/step
Epoch 4/30
30/30 - 2s - loss: 0.3869 - accuracy: 0.6677 - 2s/epoch - 75ms/step
Epoch 5/30
30/30 - 2s - loss: 0.2705 - accuracy: 0.8608 - 2s/epoch - 75ms/step
Epoch 6/30
30/30 - 2s - loss: 0.1591 - accuracy: 0.9684 - 2s/epoch - 74ms/step
Epoch 7/30
30/30 - 3s - loss: 0.1368 - accuracy: 0.9747 - 3s/epoch - 109ms/step
Epoch 8/30
30/30 - 4s - loss: 0.1271 - accuracy: 0.9800 - 4s/epoch - 132ms/step
Epoch 9/30
30/30 - 3s - loss: 0.1205 - accuracy: 0.9789 - 3s/epoch - 116ms/step
Epoch 10/30
30/30 - 2s - loss: 0.1239 - accuracy: 0.9810 - 2s/epoch - 75ms/step
Epoch 11/30
30/30 - 2s - loss: 0.1218 - accuracy: 0.9747 - 2s/epoch - 75ms/step
Epoch 12/30
30/30 - 2s - loss: 0.1133 - accuracy: 0.9789 - 2s/epoch - 75ms/step
Epoch 13/30
30/30 - 2s - loss: 0.1032 - accu

...

Epoch 1/30
30/30 - 3s - loss: 0.6582 - accuracy: 0.6744 - 3s/epoch - 111ms/step
Epoch 2/30
30/30 - 2s - loss: 0.6196 - accuracy: 0.6870 - 2s/epoch - 75ms/step
Epoch 3/30
30/30 - 2s - loss: 0.5604 - accuracy: 0.6870 - 2s/epoch - 75ms/step
Epoch 4/30
30/30 - 4s - loss: 0.3908 - accuracy: 0.7164 - 4s/epoch - 118ms/step
Epoch 5/30
30/30 - 4s - loss: 0.2009 - accuracy: 0.8666 - 4s/epoch - 132ms/step
Epoch 6/30
30/30 - 3s - loss: 0.1078 - accuracy: 0.9443 - 3s/epoch - 107ms/step
Epoch 7/30
30/30 - 2s - loss: 0.0559 - accuracy: 0.9874 - 2s/epoch - 75ms/step
Epoch 8/30
30/30 - 2s - loss: 0.0537 - accuracy: 0.9853 - 2s/epoch - 76ms/step
Epoch 9/30
30/30 - 2s - loss: 0.0427 - accuracy: 0.9821 - 2s/epoch - 76ms/step
Epoch 10/30
30/30 - 2s - loss: 0.0451 - accuracy: 0.9842 - 2s/epoch - 77ms/step
Epoch 11/30
30/30 - 4s - loss: 0.0342 - accuracy: 0.9926 - 4s/epoch - 138ms/step
Epoch 12/30
30/30 - 4s - loss: 0.0347 - accuracy: 0.9874 - 4s/epoch - 134ms/step
Epoch 13/30
30/30 - 3s - loss: 0.0394 - acc

<ipython-input-8-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
30/30 - 3s - loss: 0.6526 - accuracy: 0.6652 - 3s/epoch - 110ms/step
Epoch 2/30
30/30 - 2s - loss: 0.5821 - accuracy: 0.6706 - 2s/epoch - 78ms/step
Epoch 3/30
30/30 - 4s - loss: 0.4484 - accuracy: 0.6706 - 4s/epoch - 136ms/step
Epoch 4/30
30/30 - 4s - loss: 0.2783 - accuracy: 0.8358 - 4s/epoch - 127ms/step
Epoch 5/30
30/30 - 3s - loss: 0.1336 - accuracy: 0.9592 - 3s/epoch - 86ms/step
Epoch 6/30
30/30 - 2s - loss: 0.0748 - accuracy: 0.9710 - 2s/epoch - 73ms/step
Epoch 7/30
30/30 - 2s - loss: 0.0626 - accuracy: 0.9753 - 2s/epoch - 74ms/step
Epoch 8/30
30/30 - 2s - loss: 0.0555 - accuracy: 0.9657 - 2s/epoch - 73ms/step
Epoch 9/30
30/30 - 3s - loss: 0.0526 - accuracy: 0.9753 - 3s/epoch - 95ms/step
Epoch 10/30
30/30 - 4s - loss: 0.0563 - accuracy: 0.9775 - 4s/epoch - 131ms/step
Epoch 11/30
30/30 - 4s - loss: 0.0465 - accuracy: 0.9764 - 4s/epoch - 127ms/step
Epoch 12/30
30/30 - 2s - loss: 0.0449 - accuracy: 0.9742 - 2s/epoch - 76ms/step
Epoch 13/30
30/30 - 2s - loss: 0.0526 - accu

<ipython-input-8-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
30/30 - 5s - loss: 0.6506 - accuracy: 0.6597 - 5s/epoch - 150ms/step
Epoch 2/30
30/30 - 4s - loss: 0.6040 - accuracy: 0.6817 - 4s/epoch - 134ms/step
Epoch 3/30
30/30 - 3s - loss: 0.5105 - accuracy: 0.6817 - 3s/epoch - 109ms/step
Epoch 4/30
30/30 - 2s - loss: 0.4147 - accuracy: 0.6880 - 2s/epoch - 76ms/step
Epoch 5/30
30/30 - 2s - loss: 0.2807 - accuracy: 0.8487 - 2s/epoch - 77ms/step
Epoch 6/30
30/30 - 2s - loss: 0.1314 - accuracy: 0.9517 - 2s/epoch - 75ms/step
Epoch 7/30
30/30 - 2s - loss: 0.1085 - accuracy: 0.9643 - 2s/epoch - 75ms/step
Epoch 8/30
30/30 - 4s - loss: 0.0813 - accuracy: 0.9716 - 4s/epoch - 133ms/step
Epoch 9/30
30/30 - 4s - loss: 0.0841 - accuracy: 0.9758 - 4s/epoch - 133ms/step
Epoch 10/30
30/30 - 3s - loss: 0.0766 - accuracy: 0.9748 - 3s/epoch - 92ms/step
Epoch 11/30
30/30 - 2s - loss: 0.0707 - accuracy: 0.9821 - 2s/epoch - 75ms/step
Epoch 12/30
30/30 - 2s - loss: 0.0757 - accuracy: 0.9842 - 2s/epoch - 76ms/step
Epoch 13/30
30/30 - 2s - loss: 0.0604 - accu

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.562791,0.518061,0.520930,0.500969
1,eda2,0.611628,0.525728,0.641860,0.542995
2,eda3,0.598837,0.537585,0.654651,0.574634
3,eda4,0.632558,0.517472,0.544186,0.499869
4,eda5,0.594186,0.538387,0.582558,0.551885


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.600000,0.527447,0.588837,0.534071
std,0.025568,0.010162,0.058710,0.032817
min,0.562791,0.517472,0.520930,0.499869
25%,0.594186,0.518061,0.544186,0.500969
50%,0.598837,0.525728,0.582558,0.542995
75%,0.611628,0.537585,0.641860,0.551885
max,0.632558,0.538387,0.654651,0.574634


#GPTNEO_PKA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/teste_algo/base_offensive_'+str(perc)+'_gptneokey'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/teste_algo/base_offensive_'+str(perc)+'_gptneokey'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/teste_algo/base_offensive_'+str(perc)+'_gptneokey'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/teste_algo/base_offensive_'+str(perc)+'_gptneokey'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/teste_algo/base_offensive_'+str(perc)+'_gptneokey'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-9-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 6s - loss: 0.5911 - accuracy: 0.6637 - 6s/epoch - 172ms/step
Epoch 2/30
32/32 - 4s - loss: 0.4749 - accuracy: 0.6706 - 4s/epoch - 130ms/step
Epoch 3/30
32/32 - 3s - loss: 0.3981 - accuracy: 0.8118 - 3s/epoch - 80ms/step
Epoch 4/30
32/32 - 2s - loss: 0.2708 - accuracy: 0.9049 - 2s/epoch - 75ms/step
Epoch 5/30
32/32 - 2s - loss: 0.1419 - accuracy: 0.9471 - 2s/epoch - 76ms/step
Epoch 6/30
32/32 - 2s - loss: 0.1053 - accuracy: 0.9686 - 2s/epoch - 76ms/step
Epoch 7/30
32/32 - 4s - loss: 0.0796 - accuracy: 0.9716 - 4s/epoch - 123ms/step
Epoch 8/30
32/32 - 4s - loss: 0.0643 - accuracy: 0.9765 - 4s/epoch - 131ms/step
Epoch 9/30
32/32 - 3s - loss: 0.0693 - accuracy: 0.9676 - 3s/epoch - 93ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0509 - accuracy: 0.9863 - 2s/epoch - 76ms/step
Epoch 11/30
32/32 - 2s - loss: 0.0552 - accuracy: 0.9745 - 2s/epoch - 75ms/step
Epoch 12/30
32/32 - 2s - loss: 0.0502 - accuracy: 0.9765 - 2s/epoch - 75ms/step
Epoch 13/30
32/32 - 3s - loss: 0.0457 - accur

<ipython-input-9-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 6s - loss: 0.5708 - accuracy: 0.6755 - 6s/epoch - 174ms/step
Epoch 2/30
32/32 - 2s - loss: 0.4712 - accuracy: 0.7265 - 2s/epoch - 76ms/step
Epoch 3/30
32/32 - 2s - loss: 0.3670 - accuracy: 0.8706 - 2s/epoch - 76ms/step
Epoch 4/30
32/32 - 2s - loss: 0.2190 - accuracy: 0.9284 - 2s/epoch - 75ms/step
Epoch 5/30
32/32 - 2s - loss: 0.1505 - accuracy: 0.9588 - 2s/epoch - 76ms/step
Epoch 6/30
32/32 - 4s - loss: 0.1175 - accuracy: 0.9765 - 4s/epoch - 137ms/step
Epoch 7/30
32/32 - 4s - loss: 0.0922 - accuracy: 0.9794 - 4s/epoch - 132ms/step
Epoch 8/30
32/32 - 3s - loss: 0.0694 - accuracy: 0.9873 - 3s/epoch - 81ms/step
Epoch 9/30
32/32 - 2s - loss: 0.0825 - accuracy: 0.9863 - 2s/epoch - 75ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0805 - accuracy: 0.9882 - 2s/epoch - 75ms/step
Epoch 11/30
32/32 - 2s - loss: 0.0691 - accuracy: 0.9902 - 2s/epoch - 75ms/step
Epoch 12/30
32/32 - 4s - loss: 0.0706 - accuracy: 0.9902 - 4s/epoch - 123ms/step
Epoch 13/30
32/32 - 4s - loss: 0.0623 - accur

...

Epoch 1/30
32/32 - 6s - loss: 0.6388 - accuracy: 0.6706 - 6s/epoch - 180ms/step
Epoch 2/30
32/32 - 3s - loss: 0.5644 - accuracy: 0.6706 - 3s/epoch - 84ms/step
Epoch 3/30
32/32 - 2s - loss: 0.4406 - accuracy: 0.6833 - 2s/epoch - 76ms/step
Epoch 4/30
32/32 - 2s - loss: 0.3146 - accuracy: 0.9000 - 2s/epoch - 76ms/step
Epoch 5/30
32/32 - 2s - loss: 0.1934 - accuracy: 0.9500 - 2s/epoch - 76ms/step
Epoch 6/30
32/32 - 4s - loss: 0.1679 - accuracy: 0.9647 - 4s/epoch - 122ms/step
Epoch 7/30
32/32 - 4s - loss: 0.1354 - accuracy: 0.9676 - 4s/epoch - 132ms/step
Epoch 8/30
32/32 - 3s - loss: 0.1281 - accuracy: 0.9775 - 3s/epoch - 97ms/step
Epoch 9/30
32/32 - 2s - loss: 0.1030 - accuracy: 0.9725 - 2s/epoch - 76ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0825 - accuracy: 0.9814 - 2s/epoch - 76ms/step
Epoch 11/30
32/32 - 2s - loss: 0.0709 - accuracy: 0.9804 - 2s/epoch - 76ms/step
Epoch 12/30
32/32 - 4s - loss: 0.0797 - accuracy: 0.9843 - 4s/epoch - 110ms/step
Epoch 13/30
32/32 - 4s - loss: 0.0756 - accur

<ipython-input-9-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


32/32 - 3s - loss: 0.6468 - accuracy: 0.6637 - 3s/epoch - 109ms/step
Epoch 2/30
32/32 - 2s - loss: 0.5758 - accuracy: 0.6706 - 2s/epoch - 77ms/step
Epoch 3/30
32/32 - 4s - loss: 0.4945 - accuracy: 0.6706 - 4s/epoch - 129ms/step
Epoch 4/30
32/32 - 4s - loss: 0.4492 - accuracy: 0.6706 - 4s/epoch - 129ms/step
Epoch 5/30
32/32 - 3s - loss: 0.4008 - accuracy: 0.6706 - 3s/epoch - 89ms/step
Epoch 6/30
32/32 - 2s - loss: 0.3761 - accuracy: 0.8010 - 2s/epoch - 75ms/step
Epoch 7/30
32/32 - 2s - loss: 0.3878 - accuracy: 0.7912 - 2s/epoch - 76ms/step
Epoch 8/30
32/32 - 2s - loss: 0.3501 - accuracy: 0.8333 - 2s/epoch - 74ms/step
Epoch 9/30
32/32 - 3s - loss: 0.3735 - accuracy: 0.7941 - 3s/epoch - 109ms/step
Epoch 10/30
32/32 - 4s - loss: 0.3430 - accuracy: 0.8265 - 4s/epoch - 130ms/step
Epoch 11/30
32/32 - 3s - loss: 0.3485 - accuracy: 0.8225 - 3s/epoch - 108ms/step
Epoch 12/30
32/32 - 2s - loss: 0.3262 - accuracy: 0.8402 - 2s/epoch - 77ms/step
Epoch 13/30
32/32 - 2s - loss: 0.3418 - accuracy: 0.82

<ipython-input-9-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 4s - loss: 0.6084 - accuracy: 0.6578 - 4s/epoch - 119ms/step
Epoch 2/30
32/32 - 2s - loss: 0.4774 - accuracy: 0.6696 - 2s/epoch - 76ms/step
Epoch 3/30
32/32 - 2s - loss: 0.3642 - accuracy: 0.7814 - 2s/epoch - 77ms/step
Epoch 4/30
32/32 - 3s - loss: 0.2200 - accuracy: 0.9373 - 3s/epoch - 81ms/step
Epoch 5/30
32/32 - 4s - loss: 0.1332 - accuracy: 0.9559 - 4s/epoch - 135ms/step
Epoch 6/30
32/32 - 4s - loss: 0.0961 - accuracy: 0.9686 - 4s/epoch - 132ms/step
Epoch 7/30
32/32 - 2s - loss: 0.0978 - accuracy: 0.9755 - 2s/epoch - 76ms/step
Epoch 8/30
32/32 - 2s - loss: 0.0598 - accuracy: 0.9725 - 2s/epoch - 74ms/step
Epoch 9/30
32/32 - 2s - loss: 0.0368 - accuracy: 0.9853 - 2s/epoch - 76ms/step
Epoch 10/30
32/32 - 2s - loss: 0.0366 - accuracy: 0.9814 - 2s/epoch - 76ms/step
Epoch 11/30
32/32 - 4s - loss: 0.0391 - accuracy: 0.9804 - 4s/epoch - 127ms/step
Epoch 12/30
32/32 - 4s - loss: 0.0361 - accuracy: 0.9804 - 4s/epoch - 131ms/step
Epoch 13/30
32/32 - 3s - loss: 0.0372 - accu

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.588140,0.541699,0.626047,0.557290
std,0.085981,0.051306,0.073682,0.031574
min,0.470930,0.468757,0.517442,0.508022
25%,0.548837,0.510733,0.604651,0.542942
50%,0.593023,0.556110,0.634884,0.576726
75%,0.626744,0.582129,0.654651,0.578723
max,0.701163,0.590765,0.718605,0.580034


In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.470930,0.468757,0.517442,0.508022
1,eda2,0.593023,0.556110,0.634884,0.578723
2,eda3,0.701163,0.582129,0.718605,0.580034
3,eda4,0.626744,0.590765,0.654651,0.576726
4,eda5,0.548837,0.510733,0.604651,0.542942
